In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np



%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)

import plotly.graph_objects as go

![CRISP_DM](../reports/figures/CRISP_DM.png)

## Data Load

In [7]:
# Parsing date
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  

df_analyse.sort_values('date',ascending=True).tail()

,date,Italy,US,Spain,Germany,"Korea, South"
911,2022-07-21,20467349,90200438,13132159,30239122,19077659
912,2022-07-22,20539016,90367064,13204863,30331131,19146180
913,2022-07-23,20608190,90390185,13204863,30331133,19211613
914,2022-07-24,20660065,90410386,13204863,30331133,19247496
915,2022-07-25,20684182,90567290,13204863,30476605,19346764


## Helper functions

In [8]:
def quick_plot(x_in, df_input,y_scale='log',slider=False):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()

    for each in df_input.columns:
        fig.add_trace(go.Scatter(
                        x=x_in,
                        y=df_input[each],
                        name=each,
                        opacity=0.8))
    
    fig.update_layout(autosize=True,
        width=1024,
        height=768,
        font=dict(
            family="PT Sans, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y_scale),
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()
      

In [9]:
quick_plot(df_analyse.date,
           df_analyse.iloc[:,1:],
           y_scale='log',
           slider=True)

In [10]:
threshold=100

In [11]:
compare_list=[]
for pos,country in enumerate(df_analyse.columns[1:]):
    compare_list.append(np.array(df_analyse[country][df_analyse[country]>threshold]))
compare_list

[array([     155,      229,      322,      453,      655,      888,
            1128,     1694,     2036,     2502,     3089,     3858,
            4636,     5883,     7375,     9172,    10149,    12462,
           15113,    17660,    21157,    24747,    27980,    31506,
           35713,    41035,    47021,    53578,    59138,    63927,
           69176,    74386,    80589,    86498,    92472,    97689,
          101739,   105792,   110574,   115242,   119827,   124632,
          128948,   132547,   135586,   139422,   143626,   147577,
          152271,   156363,   159516,   162488,   165155,   168941,
          172434,   175925,   178972,   181228,   183957,   187327,
          189973,   192994,   195351,   197675,   199414,   201505,
          203591,   205463,   207428,   209328,   210717,   211938,
          213013,   214457,   215858,   217185,   218268,   219070,
          219814,   221216,   222104,   223096,   223885,   224760,
          225435,   225886,   226699,   227364, 

In [12]:
pd_sync_timelines=pd.DataFrame(compare_list,index=df_analyse.columns[1:]).T

In [13]:
pd_sync_timelines['date']=np.arange(pd_sync_timelines.shape[0])

In [14]:
pd_sync_timelines.head()

,Italy,US,Spain,Germany,"Korea, South",date
0,155.0,107.0,120.0,117.0,104.0,0
1,229.0,184.0,165.0,150.0,204.0,1
2,322.0,237.0,222.0,188.0,433.0,2
3,453.0,403.0,259.0,240.0,602.0,3
4,655.0,519.0,400.0,349.0,833.0,4


In [15]:
quick_plot(pd_sync_timelines.date,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True)

## Doubling Rate

$N(t) = N_0* 2^(t/T)$

In [16]:
def doubling_rate(N_0,t,T_d):
    return N_0*np.power(2,t/T_d)

In [17]:
max_days=34

norm_slopes={
    #'doubling every day':doubling_rate(100,np.arange(10),1),
    'doubling every two days':doubling_rate(100,np.arange(max_days),2),
    'doubling every 4 days':doubling_rate(100,np.arange(max_days),4),
    'doubling every 10 days':doubling_rate(100,np.arange(max_days),10),
}

In [18]:
pd_sync_timelines_w_slope=pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines], axis=1)

In [19]:
pd_sync_timelines_w_slope

,doubling every two days,doubling every 4 days,doubling every 10 days,Italy,US,Spain,Germany,"Korea, South",date
0,100.000000,100.000000,100.000000,155.0,107.0,120.0,117.0,104.0,0
1,141.421356,118.920712,107.177346,229.0,184.0,165.0,150.0,204.0,1
2,200.000000,141.421356,114.869835,322.0,237.0,222.0,188.0,433.0,2
3,282.842712,168.179283,123.114441,453.0,403.0,259.0,240.0,602.0,3
4,400.000000,200.000000,131.950791,655.0,519.0,400.0,349.0,833.0,4
...,...,...,...,...,...,...,...,...,...
882,NaN,NaN,NaN,20660065.0,NaN,NaN,NaN,19077659.0,882
883,NaN,NaN,NaN,20684182.0,NaN,NaN,NaN,19146180.0,883
884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19211613.0,884
885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19247496.0,885


In [20]:
pd_sync_timelines_w_slope

,doubling every two days,doubling every 4 days,doubling every 10 days,Italy,US,Spain,Germany,"Korea, South",date
0,100.000000,100.000000,100.000000,155.0,107.0,120.0,117.0,104.0,0
1,141.421356,118.920712,107.177346,229.0,184.0,165.0,150.0,204.0,1
2,200.000000,141.421356,114.869835,322.0,237.0,222.0,188.0,433.0,2
3,282.842712,168.179283,123.114441,453.0,403.0,259.0,240.0,602.0,3
4,400.000000,200.000000,131.950791,655.0,519.0,400.0,349.0,833.0,4
...,...,...,...,...,...,...,...,...,...
882,NaN,NaN,NaN,20660065.0,NaN,NaN,NaN,19077659.0,882
883,NaN,NaN,NaN,20684182.0,NaN,NaN,NaN,19146180.0,883
884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19211613.0,884
885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19247496.0,885


In [21]:
quick_plot(pd_sync_timelines_w_slope.date,
           pd_sync_timelines_w_slope.iloc[:,0:5],
           y_scale='log',
           slider=True)

In [22]:
pd_sync_timelines_w_slope.to_csv('../data/processed/COVID_small_sync_timeline_table.csv',sep=';',index=False)

## Linear Regression

In [23]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=False)

In [24]:
l_vec=len(df_analyse['Germany'])
X=np.arange(l_vec-5).reshape(-1, 1)
y=np.log(np.array(df_analyse['Germany'][5:]))

In [25]:
reg.fit(X,y)

LinearRegression(fit_intercept=False)

In [26]:
X_hat=np.arange(l_vec).reshape(-1, 1)
Y_hat=reg.predict(X_hat)

In [27]:
LR_inspect=df_analyse[['date','Germany']].copy()

In [28]:
LR_inspect['prediction']=np.exp(Y_hat)

In [29]:
quick_plot(LR_inspect.date,
           LR_inspect.iloc[:,1:],
           y_scale='log',
           slider=True)

## Piecewise Linear Regression

In [30]:
country_list=df_analyse.columns[1:]
reg = linear_model.LinearRegression(fit_intercept=True)
l_vec=len(df_analyse['Germany'])
X=np.arange(l_vec-50).reshape(-1, 1)
y=np.log(np.array(df_analyse['Germany'][50:]))

In [31]:
reg.fit(X,y)

LinearRegression()

In [32]:
reg.intercept_

11.53916838778522

In [33]:
reg.coef_

array([0.00700474])

In [34]:
def get_doubling_time_via_regression(in_array):
    ''' Using linear regression to approximate the doubling rate'''
    
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

In [35]:
def doubling_time(in_array):
    ''' Use a classical doubling time formular, 
     see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [36]:
# calculating slope of regression of last x days
# using a limited number of days to approximate the triangle, giving attention exponential base assumption
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(country_list):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                        window=3,
                        min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)
df_analyse

,date,Italy,US,Spain,Germany,"Korea, South",Italy_DR,US_DR,Spain_DR,Germany_DR,"Korea, South_DR"
0,2020-01-22,0,1,0,0,1,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,0,1,0,0,1,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,0,2,0,0,2,NaN,2.666667,NaN,NaN,2.666667
3,2020-01-25,0,2,0,0,2,NaN,3.333333,NaN,NaN,3.333333
4,2020-01-26,0,5,0,0,3,NaN,2.000000,NaN,NaN,4.666667
...,...,...,...,...,...,...,...,...,...,...,...
911,2022-07-21,20467349,90200438,13132159,30239122,19077659,242.182999,486.703188,inf,2.464517e+02,272.152750
912,2022-07-22,20539016,90367064,13204863,30331131,19146180,267.151338,562.367482,361.916639,3.025988e+02,278.302548
913,2022-07-23,20608190,90390185,13204863,30331133,19211613,291.650656,951.996385,362.583306,6.586269e+02,285.846644
914,2022-07-24,20660065,90410386,13204863,30331133,19247496,340.398081,4172.901143,inf,3.033113e+07,379.047001


In [38]:
quick_plot(df_analyse.date,
           df_analyse.iloc[:,[6,7,8,9]],
           y_scale='log',
           slider=True)

In [39]:
def doubling_time(in_array):
    ''' Use a classical doubling time formular, 
     see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [40]:
# calculating slope of regression of last x days
# using a limited number of days to approximate the triangle, giving attention exponential base assumption
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(country_list):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                        window=3,
                        min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)
df_analyse

,date,Italy,US,Spain,Germany,"Korea, South",Italy_DR,US_DR,Spain_DR,Germany_DR,"Korea, South_DR"
0,2020-01-22,0,1,0,0,1,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,0,1,0,0,1,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,0,2,0,0,2,NaN,2.666667,NaN,NaN,2.666667
3,2020-01-25,0,2,0,0,2,NaN,3.333333,NaN,NaN,3.333333
4,2020-01-26,0,5,0,0,3,NaN,2.000000,NaN,NaN,4.666667
...,...,...,...,...,...,...,...,...,...,...,...
911,2022-07-21,20467349,90200438,13132159,30239122,19077659,242.182999,486.703188,inf,2.464517e+02,272.152750
912,2022-07-22,20539016,90367064,13204863,30331131,19146180,267.151338,562.367482,361.916639,3.025988e+02,278.302548
913,2022-07-23,20608190,90390185,13204863,30331133,19211613,291.650656,951.996385,362.583306,6.586269e+02,285.846644
914,2022-07-24,20660065,90410386,13204863,30331133,19247496,340.398081,4172.901143,inf,3.033113e+07,379.047001


In [42]:
quick_plot(df_analyse.date,
           df_analyse.iloc[:,[6,7,9,10]],
           y_scale='log',
           slider=True)

In [43]:
from scipy import signal

## filter data
for each in country_list:
#     filter_in=df_analyse[each].fillna(0) # attention with the neutral element here 
    df_analyse[each+'_filter']=signal.savgol_filter(df_analyse[each],
                           5, # window size used for filtering
                           1) # order of fitted polynomial   
filter_cols=['Italy_filter','US_filter', 'Spain_filter', 'Germany_filter']

In [44]:
# run on all filtered data
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR_filter']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)
df_analyse

,date,Italy,US,Spain,Germany,"Korea, South",Italy_DR,US_DR,Spain_DR,Germany_DR,"Korea, South_DR",Italy_filter,US_filter,Spain_filter,Germany_filter,"Korea, South_filter",Italy_filter_DR_filter,US_filter_DR_filter,Spain_filter_DR_filter,Germany_filter_DR_filter
0,2020-01-22,0,1,0,0,1,NaN,NaN,NaN,NaN,NaN,0.0,0.4,0.0,0.0,0.8,NaN,NaN,NaN,NaN
1,2020-01-23,0,1,0,0,1,NaN,NaN,NaN,NaN,NaN,0.0,1.3,0.0,0.0,1.3,NaN,NaN,NaN,NaN
2,2020-01-24,0,2,0,0,2,NaN,2.666667,NaN,NaN,2.666667,0.0,2.2,0.0,0.0,1.8,NaN,1.444444,NaN,NaN
3,2020-01-25,0,2,0,0,2,NaN,3.333333,NaN,NaN,3.333333,0.0,3.0,0.0,0.2,2.4,NaN,2.549020,NaN,0.666667
4,2020-01-26,0,5,0,0,3,NaN,2.000000,NaN,NaN,4.666667,0.0,3.8,0.0,1.0,3.0,NaN,3.750000,NaN,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,2022-07-21,20467349,90200438,13132159,30239122,19077659,242.182999,486.703188,inf,2.464517e+02,272.152750,20459876.4,90166889.0,13161240.6,30205473.6,19076500.6,247.454210,601.188637,574.294422,269.804637
912,2022-07-22,20539016,90367064,13204863,30331131,19146180,267.151338,562.367482,361.916639,3.025988e+02,278.302548,20532086.8,90282866.8,13175781.4,30272764.4,19138405.6,258.491521,716.792257,703.318799,371.001523
913,2022-07-23,20608190,90390185,13204863,30331133,19211613,291.650656,951.996385,362.583306,6.586269e+02,285.846644,20591760.4,90387072.6,13190322.2,30341824.8,19205942.4,311.302476,820.033307,906.124931,444.049693
914,2022-07-24,20660065,90410386,13204863,30331133,19247496,340.398081,4172.901143,inf,3.033113e+07,379.047001,20647231.9,90464775.2,13204863.0,30389321.6,19269895.0,357.641961,993.667562,907.124931,520.510735


In [46]:
start_pos = 40
quick_plot(df_analyse.date[start_pos:],
           df_analyse.iloc[start_pos:,[9,19]],
           y_scale='log',
           slider=True)